# Kokoro TTS Local - Synthese Vocale Legere

**Module :** 01-Audio-Foundation  
**Niveau :** Intermediaire  
**Technologies :** Kokoro TTS (82M params, MIT license)  
**Duree estimee :** 35 minutes  
**VRAM :** ~2 GB  

## Objectifs d'Apprentissage

- [ ] Installer et charger le modele Kokoro depuis HuggingFace
- [ ] Comprendre l'architecture et les avantages de Kokoro (82M params, MIT)
- [ ] Generer de la parole avec differentes voix et langues
- [ ] Comparer la qualite avec OpenAI TTS (evaluation subjective MOS-like)
- [ ] Mesurer la latence et le debit de generation
- [ ] Analyser le cout : local gratuit vs API payante
- [ ] Identifier quand utiliser le TTS local vs cloud

## Prerequis

- GPU NVIDIA avec au moins 2 GB VRAM (ou CPU)
- `pip install kokoro-onnx` ou `pip install kokoro`
- Notebook 01-1 recommande (pour la comparaison avec OpenAI TTS)

**Navigation :** [Index](../README.md) | [<< Precedent](01-4-Whisper-Local.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres Kokoro
kokoro_model = "hexgrad/Kokoro-82M"   # Modele HuggingFace
voice = "af_heart"                    # Voix par defaut
use_onnx = True                       # Utiliser le backend ONNX (plus rapide)

# Configuration
compare_with_openai = True         # Comparer avec OpenAI TTS
test_multiple_voices = True        # Tester plusieurs voix
benchmark_latency = True           # Benchmarks de latence
save_results = True                # Sauvegarder les resultats

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
import gc
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import logging

import numpy as np
import soundfile as sf
from IPython.display import Audio, display

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import play_audio, save_audio
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'kokoro'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('kokoro_tts')

# Verification GPU
gpu_available = False
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_vram = torch.cuda.get_device_properties(0).total_mem / (1024**3)
        print(f"GPU : {gpu_name} ({gpu_vram:.1f} GB VRAM)")
    else:
        print("GPU non disponible - Kokoro fonctionne aussi sur CPU")
except ImportError:
    print("torch non installe - utilisation CPU")

print(f"\nKokoro TTS Local - Synthese Vocale")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele : {kokoro_model}")
print(f"Voix : {voice}")
print(f"Backend ONNX : {use_onnx}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes


GPU non disponible - Kokoro fonctionne aussi sur CPU

Kokoro TTS Local - Synthese Vocale
Date : 2026-02-18 10:28:45
Modele : hexgrad/Kokoro-82M
Voix : af_heart
Backend ONNX : True
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\kokoro


In [3]:
# Chargement .env (pour comparaison OpenAI)
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

openai_key = os.getenv('OPENAI_API_KEY')
if openai_key:
    print(f"OPENAI_API_KEY disponible (pour comparaison)")
else:
    print(f"OPENAI_API_KEY non disponible - comparaison avec OpenAI desactivee")
    compare_with_openai = False

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env
OPENAI_API_KEY disponible (pour comparaison)


## Section 1 : Presentation de Kokoro

Kokoro est un modele TTS leger et open-source sous licence MIT.

### Caracteristiques

| Aspect | Kokoro 82M | OpenAI TTS |
|--------|-----------|------------|
| Parametres | 82 millions | Non publie |
| Licence | MIT (libre) | Proprietaire |
| Cout | Gratuit | $15-30 / 1M chars |
| Hebergement | Local | Cloud OpenAI |
| VRAM | ~2 GB | N/A (API) |
| Latence | ~0.5-1s (GPU) | ~1-3s (reseau) |
| Langues | EN, FR, ES, ZH, JA, KO | Multilingue |
| Qualite | Bonne (MOS ~4.0) | Excellente (MOS ~4.5) |

### Quand utiliser Kokoro vs OpenAI

| Scenario | Recommandation | Raison |
|----------|---------------|--------|
| Prototypage | Kokoro | Gratuit, pas de cle API |
| Production web | OpenAI TTS | Qualite superieure |
| Donnees sensibles | Kokoro | Execution locale |
| Volume eleve | Kokoro | Pas de cout par caractere |
| Meilleure qualite | OpenAI TTS-HD | Voix les plus naturelles |

In [4]:
# Chargement du modele Kokoro
print("CHARGEMENT DU MODELE KOKORO")
print("=" * 45)

kokoro_loaded = False

if use_onnx:
    try:
        from kokoro_onnx import Kokoro
        from pathlib import Path
        import requests
        
        print("Chargement Kokoro (backend ONNX)...")
        
        # Chemins pour les fichiers du modele
        cache_dir = Path.home() / '.cache' / 'kokoro-onnx'
        cache_dir.mkdir(parents=True, exist_ok=True)
        
        model_path = cache_dir / 'kokoro-v1.0.onnx'
        voices_path = cache_dir / 'voices-v1.0.bin'
        
        # Telecharger si necessaire
        if not model_path.exists():
            print("Telechargement du modele...")
            response = requests.get('https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/kokoro-v1.0.onnx', stream=True)
            with open(model_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"Modele telecharge : {model_path.stat().st_size / (1024*1024):.1f} MB")
        
        if not voices_path.exists():
            print("Telechargement des voix...")
            response = requests.get('https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/voices-v1.0.bin', stream=True)
            with open(voices_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"Voices telechargees : {voices_path.stat().st_size / (1024*1024):.1f} MB")
        
        start_time = time.time()
        kokoro = Kokoro(str(model_path), str(voices_path))
        load_time = time.time() - start_time
        kokoro_loaded = True
        
        available_voices = kokoro.get_voices()
        print(f"Modele charge en {load_time:.1f}s (ONNX)")
        print(f"Voix disponibles ({len(available_voices)}) : {', '.join(available_voices[:10])}{'...' if len(available_voices) > 10 else ''}")
        
    except ImportError:
        print("kokoro-onnx non installe, tentative avec kokoro standard...")
        use_onnx = False
    except Exception as e:
        print(f"Erreur chargement ONNX: {str(e)[:100]}")
        print("Tentative avec kokoro standard...")
        use_onnx = False

if not use_onnx and not kokoro_loaded:
    try:
        from kokoro import KPipeline
        print("Chargement Kokoro (backend PyTorch)...")
        start_time = time.time()
        pipeline = KPipeline(lang_code='a')  # 'a' = auto
        load_time = time.time() - start_time
        kokoro_loaded = True
        print(f"Modele charge en {load_time:.1f}s (PyTorch)")
    except ImportError:
        print("Ni kokoro-onnx ni kokoro ne sont installes")
        print("Installation : pip install kokoro-onnx")
        print("Ou : pip install kokoro soundfile")

if kokoro_loaded:
    print(f"\nModele pret pour la generation")
    if gpu_available:
        vram_used = torch.cuda.memory_allocated(0) / (1024**3)
        print(f"VRAM utilisee : {vram_used:.2f} GB")

CHARGEMENT DU MODELE KOKORO
kokoro-onnx non installe, tentative avec kokoro standard...
Ni kokoro-onnx ni kokoro ne sont installes
Installation : pip install kokoro-onnx
Ou : pip install kokoro soundfile


## Section 2 : Premiere generation

La generation avec Kokoro suit un pipeline simple :
1. Charger le modele (fait une seule fois)
2. Specifier le texte et la voix
3. Appeler la methode de generation
4. Recevoir un array numpy + sample rate

In [5]:
# Premiere generation TTS avec Kokoro
print("PREMIERE GENERATION KOKORO")
print("=" * 45)

sample_text = (
    "Bonjour et bienvenue dans ce cours sur la synthese vocale locale. "
    "Kokoro est un modele leger et open-source qui fonctionne directement "
    "sur votre machine."
)

print(f"Texte : {sample_text}")
print(f"Voix : {voice}")

if kokoro_loaded:
    start_time = time.time()

    if use_onnx:
        # API kokoro-onnx
        samples, sample_rate = kokoro.create(
            sample_text,
            voice=voice,
            speed=1.0
        )
    else:
        # API kokoro standard (KPipeline)
        generator = pipeline(
            sample_text,
            voice=voice,
            speed=1.0
        )
        # Concatener tous les segments
        all_samples = []
        sample_rate = 24000
        for gs, ps, audio_chunk in generator:
            all_samples.append(audio_chunk)
        samples = np.concatenate(all_samples) if all_samples else np.array([])

    gen_time = time.time() - start_time
    duration = len(samples) / sample_rate

    print(f"\nGeneration reussie")
    print(f"  Duree audio : {duration:.1f}s")
    print(f"  Sample rate : {sample_rate} Hz")
    print(f"  Temps de generation : {gen_time:.2f}s")
    print(f"  Ratio temps reel : {duration / gen_time:.1f}x")
    print(f"  Taille : {len(samples) * 4 / 1024:.1f} KB (float32)")

    # Lecture
    print(f"\nEcoute :")
    display(Audio(data=samples, rate=sample_rate))

    # Sauvegarde
    if save_results:
        filepath = OUTPUT_DIR / f"kokoro_{voice}.wav"
        sf.write(str(filepath), samples, sample_rate)
        print(f"Fichier sauvegarde : {filepath.name}")
else:
    print("Modele Kokoro non charge")

PREMIERE GENERATION KOKORO
Texte : Bonjour et bienvenue dans ce cours sur la synthese vocale locale. Kokoro est un modele leger et open-source qui fonctionne directement sur votre machine.
Voix : af_heart
Modele Kokoro non charge


### Interpretation : Premiere generation

| Aspect | Valeur typique | Signification |
|--------|---------------|---------------|
| Ratio temps reel | 5-20x (GPU), 1-3x (CPU) | Kokoro genere bien plus vite que le temps reel |
| Sample rate | 24000 Hz | Standard pour la parole (qualite telephone HD) |
| Qualite | Bonne | Moins naturelle qu'OpenAI TTS mais acceptable |

> **Note technique** : Le premier appel est plus lent (compilation JIT). Les appels suivants beneficient du cache.

## Section 3 : Voix et langues

Kokoro propose plusieurs voix organisees par langue :

| Prefixe | Langue | Exemples de voix |
|---------|--------|------------------|
| `af_` | Anglais (feminin) | `af_heart`, `af_bella`, `af_sarah` |
| `am_` | Anglais (masculin) | `am_adam`, `am_michael` |
| `bf_` | Anglais britannique (f) | `bf_emma`, `bf_isabella` |
| `bm_` | Anglais britannique (m) | `bm_george`, `bm_lewis` |
| `ff_` | Francais (f) | `ff_siwis` |
| `fm_` | Francais (m) | (a venir) |

In [6]:
# Test de differentes voix
print("TEST DES VOIX")
print("=" * 45)

if kokoro_loaded and test_multiple_voices:
    voices_to_test = [
        ("af_heart", "Hello, I am the Heart voice from Kokoro."),
        ("af_bella", "Hello, I am the Bella voice from Kokoro."),
        ("am_adam", "Hello, I am the Adam voice from Kokoro."),
        ("ff_siwis", "Bonjour, je suis la voix Siwis de Kokoro."),
    ]

    voice_results = {}

    for v, text in voices_to_test:
        print(f"\nVoix : {v}")
        try:
            start_time = time.time()

            if use_onnx:
                samples, sample_rate = kokoro.create(text, voice=v, speed=1.0)
            else:
                generator = pipeline(text, voice=v, speed=1.0)
                all_samples = []
                sample_rate = 24000
                for gs, ps, audio_chunk in generator:
                    all_samples.append(audio_chunk)
                samples = np.concatenate(all_samples) if all_samples else np.array([])

            gen_time = time.time() - start_time
            duration = len(samples) / sample_rate

            voice_results[v] = {
                "duration": duration,
                "gen_time": gen_time,
                "samples": samples,
                "sample_rate": sample_rate
            }

            print(f"  Duree : {duration:.1f}s | Temps : {gen_time:.2f}s | Ratio : {duration/gen_time:.1f}x")
            display(Audio(data=samples, rate=sample_rate))

            if save_results:
                filepath = OUTPUT_DIR / f"voice_{v}.wav"
                sf.write(str(filepath), samples, sample_rate)

        except Exception as e:
            print(f"  Erreur : {str(e)[:80]}")

    # Tableau recapitulatif
    if voice_results:
        print(f"\nRecapitulatif :")
        print(f"{'Voix':<15} {'Duree':<10} {'Temps gen':<12} {'Ratio':<8}")
        print("-" * 45)
        for v, data in voice_results.items():
            ratio = data['duration'] / data['gen_time'] if data['gen_time'] > 0 else 0
            print(f"{v:<15} {data['duration']:<10.1f} {data['gen_time']:<12.2f} {ratio:<8.1f}")
else:
    print("Test des voix desactive ou modele non charge")

TEST DES VOIX
Test des voix desactive ou modele non charge


## Section 4 : Comparaison avec OpenAI TTS

Comparaison directe entre Kokoro (local, gratuit) et OpenAI TTS (cloud, payant) sur les memes textes.

### Criteres d'evaluation

| Critere | Description |
|---------|-------------|
| Latence | Temps entre la requete et le debut de l'audio |
| Naturalite | Ressemblance avec la voix humaine |
| Prosodie | Rythme, intonation, accentuation |
| Cout | Prix par caractere synthetise |

In [7]:
# Comparaison Kokoro vs OpenAI TTS
print("COMPARAISON KOKORO VS OPENAI TTS")
print("=" * 45)

comparison_text = (
    "L'intelligence artificielle generative transforme la facon "
    "dont nous creons et consommons du contenu audio."
)

comparison_results = {}

# --- Kokoro ---
if kokoro_loaded:
    print(f"\n--- Kokoro ({voice}) ---")
    start_time = time.time()

    if use_onnx:
        kokoro_samples, kokoro_sr = kokoro.create(comparison_text, voice=voice, speed=1.0)
    else:
        generator = pipeline(comparison_text, voice=voice, speed=1.0)
        all_samples = []
        kokoro_sr = 24000
        for gs, ps, audio_chunk in generator:
            all_samples.append(audio_chunk)
        kokoro_samples = np.concatenate(all_samples) if all_samples else np.array([])

    kokoro_time = time.time() - start_time
    kokoro_duration = len(kokoro_samples) / kokoro_sr

    comparison_results["Kokoro"] = {
        "time": kokoro_time,
        "duration": kokoro_duration,
        "cost": 0.0
    }

    print(f"  Temps : {kokoro_time:.2f}s")
    print(f"  Duree audio : {kokoro_duration:.1f}s")
    print(f"  Cout : $0.00")
    display(Audio(data=kokoro_samples, rate=kokoro_sr))

# --- OpenAI TTS ---
if compare_with_openai:
    from openai import OpenAI
    client_api = OpenAI(api_key=openai_key)

    for tts_model in ["tts-1", "tts-1-hd"]:
        print(f"\n--- OpenAI {tts_model} (alloy) ---")
        start_time = time.time()

        response = client_api.audio.speech.create(
            model=tts_model,
            voice="alloy",
            input=comparison_text,
            response_format="wav"
        )

        openai_time = time.time() - start_time
        openai_bytes = response.content

        # Charger pour connaitre la duree
        import io
        openai_data, openai_sr = sf.read(io.BytesIO(openai_bytes))
        openai_duration = len(openai_data) / openai_sr

        cost_per_char = 0.015 if tts_model == "tts-1" else 0.030
        cost = len(comparison_text) * cost_per_char / 1000

        comparison_results[f"OpenAI {tts_model}"] = {
            "time": openai_time,
            "duration": openai_duration,
            "cost": cost
        }

        print(f"  Temps : {openai_time:.2f}s")
        print(f"  Duree audio : {openai_duration:.1f}s")
        print(f"  Cout : ${cost:.5f}")
        display(Audio(data=openai_bytes, autoplay=False))

# --- Tableau comparatif ---
if comparison_results:
    print(f"\n{'='*60}")
    print(f"TABLEAU COMPARATIF")
    print(f"{'='*60}")
    print(f"{'Service':<22} {'Latence (s)':<14} {'Duree (s)':<12} {'Cout ($)':<10}")
    print("-" * 58)
    for name, data in comparison_results.items():
        print(f"{name:<22} {data['time']:<14.2f} {data['duration']:<12.1f} {data['cost']:<10.5f}")

    # Estimation pour 1 heure de narration (~50K caracteres)
    chars_1h = 50000
    print(f"\nEstimation pour 1 heure de narration (~{chars_1h:,} caracteres) :")
    print(f"  Kokoro          : $0.00")
    print(f"  OpenAI tts-1    : ${chars_1h * 0.015 / 1000:.2f}")
    print(f"  OpenAI tts-1-hd : ${chars_1h * 0.030 / 1000:.2f}")

COMPARAISON KOKORO VS OPENAI TTS



--- OpenAI tts-1 (alloy) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Temps : 3.04s
  Duree audio : 6.5s
  Cout : $0.00161



--- OpenAI tts-1-hd (alloy) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Temps : 2.48s
  Duree audio : 6.7s
  Cout : $0.00321



TABLEAU COMPARATIF
Service                Latence (s)    Duree (s)    Cout ($)  
----------------------------------------------------------
OpenAI tts-1           3.04           6.5          0.00161   
OpenAI tts-1-hd        2.48           6.7          0.00321   

Estimation pour 1 heure de narration (~50,000 caracteres) :
  Kokoro          : $0.00
  OpenAI tts-1    : $0.75
  OpenAI tts-1-hd : $1.50


### Interpretation : Comparaison Kokoro vs OpenAI

| Critere | Kokoro | OpenAI tts-1 | OpenAI tts-1-hd |
|---------|--------|-------------|------------------|
| Latence | ~0.5s (GPU) | ~1-2s (reseau) | ~2-3s (reseau) |
| Naturalite | Bonne | Tres bonne | Excellente |
| Cout/1h | $0 | ~$0.75 | ~$1.50 |
| Confidentialite | Locale | Cloud | Cloud |

**Points cles** :
1. Kokoro est imbattable sur le cout pour les gros volumes
2. OpenAI offre une meilleure qualite perceptuelle, surtout en tts-1-hd
3. Pour les donnees sensibles, Kokoro est le seul choix viable
4. En termes de latence, Kokoro avec GPU est competitif

In [8]:
# Mode interactif
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF")
    print("=" * 50)
    print("\nEntrez un texte a synthetiser avec Kokoro :")
    print("(Laissez vide pour passer a la suite)")

    try:
        user_text = input("\nVotre texte : ")

        if user_text.strip() and kokoro_loaded:
            user_voice = input(f"Voix [{voice}] : ").strip() or voice

            print(f"\nGeneration avec Kokoro ({user_voice})...")
            start_time = time.time()

            if use_onnx:
                samples, sr_out = kokoro.create(user_text, voice=user_voice, speed=1.0)
            else:
                generator = pipeline(user_text, voice=user_voice, speed=1.0)
                all_samples = []
                sr_out = 24000
                for gs, ps, audio_chunk in generator:
                    all_samples.append(audio_chunk)
                samples = np.concatenate(all_samples) if all_samples else np.array([])

            gen_time = time.time() - start_time
            print(f"Duree : {len(samples)/sr_out:.1f}s | Temps : {gen_time:.2f}s")
            display(Audio(data=samples, rate=sr_out))

            if save_results:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                filepath = OUTPUT_DIR / f"custom_{user_voice}_{ts}.wav"
                sf.write(str(filepath), samples, sr_out)
                print(f"Sauvegarde : {filepath.name}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF

Entrez un texte a synthetiser avec Kokoro :
(Laissez vide pour passer a la suite)
Mode interactif non disponible (execution automatisee)


## Bonnes pratiques et guide de decision

### Arbre de decision : local vs cloud

| Question | Oui -> | Non -> |
|----------|--------|--------|
| Donnees sensibles ? | Local (Kokoro) | Continuer |
| Budget limite ? | Local (Kokoro) | Continuer |
| Qualite maximale requise ? | Cloud (OpenAI HD) | Continuer |
| GPU disponible ? | Local (Kokoro) | Cloud (OpenAI tts-1) |
| Volume > 1M chars/mois ? | Local (Kokoro) | Cloud (OpenAI tts-1) |

### Optimisation Kokoro

| Technique | Impact | Description |
|-----------|--------|-------------|
| Backend ONNX | Vitesse +30-50% | Inference optimisee sans PyTorch |
| Batch generation | Debit +50% | Regrouper les textes courts |
| GPU | Vitesse 5-10x vs CPU | Recommande pour la production |

In [9]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele : {kokoro_model}")
print(f"Backend : {'ONNX' if use_onnx else 'PyTorch'}")
print(f"Voix par defaut : {voice}")
print(f"Modele charge : {'Oui' if kokoro_loaded else 'Non'}")

if gpu_available:
    vram_current = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"VRAM utilisee : {vram_current:.2f} GB")

if save_results:
    saved = list(OUTPUT_DIR.glob('*'))
    total_size = sum(f.stat().st_size for f in saved) / (1024*1024)
    print(f"Fichiers sauvegardes : {len(saved)} ({total_size:.1f} MB) dans {OUTPUT_DIR}")

# Liberation memoire
if kokoro_loaded:
    print(f"\nLiberation du modele...")
    if use_onnx:
        del kokoro
    else:
        del pipeline
    gc.collect()
    if gpu_available:
        torch.cuda.empty_cache()
    print(f"Memoire liberee")

print(f"\nPROCHAINES ETAPES")
print(f"1. Explorer le TTS avance avec Chatterbox (02-1)")
print(f"2. Decouvrir le voice cloning avec XTTS (02-2)")
print(f"3. Comparer tous les modeles TTS et STT (03-1)")
print(f"4. Construire un pipeline vocal complet (03-2)")

print(f"\nNotebook Kokoro TTS termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 10:28:52
Modele : hexgrad/Kokoro-82M
Backend : PyTorch
Voix par defaut : af_heart
Modele charge : Non
Fichiers sauvegardes : 5 (1.0 MB) dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\kokoro

PROCHAINES ETAPES
1. Explorer le TTS avance avec Chatterbox (02-1)
2. Decouvrir le voice cloning avec XTTS (02-2)
3. Comparer tous les modeles TTS et STT (03-1)
4. Construire un pipeline vocal complet (03-2)

Notebook Kokoro TTS termine - 10:28:52
